<img src="idal-logo.png" align="right" style="float" width="400">
<font color="#CA3532"><h1 align="left">mIA3</h1></font>
<font color="#6E6E6E"><h2 align="left">Tarea Evaluable. Aprendizaje Profundo 1 y 2 (Parte 3 de 3).</h2></font> 

#### Elaborado por Felipe Ramírez Herrera

In [ ]:
# !pip install numpy
# !pip install panda

# !pip install torch
# !pip install torchtext
# !pip install torchmetrics
# !pip install spacy

# !pip install jupyter
# !pip install ipywidgets

# !pip install editdistance
# !pip install six
# !pip install typeguard

# !pip install matplotlib
# !pip install seaborn
# !pip install scikit-learn

# !python -m spacy download en_core_web_sm
# !python -m spacy download es_core_news_sm
# !python -m spacy download fr_core_news_sm
# !pip install wget

# !pip install --upgrade --force-reinstall torchtext

In [ ]:
import math
import random
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
import numpy as np
import os
import pathlib
import csv
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torchtext.vocab import Vocab
import logging
import pandas as pd
import six
from typing import List, Tuple, Union
from argparse import Namespace
from tqdm.notebook import tqdm

import matplotlib.pyplot as mp
from matplotlib.colors import ListedColormap
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split

import cProfile, pstats, io
from pstats import SortKey
warnings.filterwarnings('ignore')

In [ ]:
# Formato de Mathplotlib

mp.rc('font', size=8)
mp.rc('axes', titlesize=8)
mp.rc('axes', labelsize=8)
mp.rc('xtick', labelsize=8)
mp.rc('ytick', labelsize=8)
mp.rc('legend', fontsize=8)
mp.rc('figure', titlesize=8)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [ ]:

torch.manual_seed(1234)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    !nvidia-smi

In [ ]:
def get_cuda_memory_available():
    if torch.cuda.is_available():
        t = torch.cuda.get_device_properties(DEVICE).total_memory
        r = torch.cuda.memory_reserved(DEVICE)
        a = torch.cuda.memory_allocated(DEVICE)
        f = (t - a - r) / t  # free inside reserved
        print("CUDA Available Memory: {0}".format(f))
    

In [ ]:
def count_params(model, return_int=False):
    params = sum([torch.prod(torch.tensor(x.shape)).item() for x in model.parameters() if x.requires_grad])
    if return_int:
        return params
    else:
        print("There are {:,} trainable parameters in this model.".format(params))

In [ ]:
# ELIMINAR

# import re
# import unicodedata

# def unicodeToAscii(s):
#     return ''.join(
#       c for c in unicodedata.normalize('NFD', s) 
#       if unicodedata.category(c) != 'Mn'
#     )

# def normalizeString(s):
#     s = unicodeToAscii(s.lower().strip())
#     s = re.sub(r"[^a-zA-Z.!?]+", " ", s)
#     return s


In [ ]:
# https://conferences.unite.un.org/UNCorpus/Home/DownloadOverview
# !cat /home/framirez/translation_multilingual/UNv1.0.6way.tar.gz.* | tar -xzf -

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import zipfile

# EN_corpus_file = "UNv1.0.6way_en.zip" # UN Parallel Corpus English (EN)
# ES_corpus_file = "UNv1.0.6way_es.zip" # UN Parallel Corpus Spanish (ES)
# FR_corpus_file = "UNv1.0.6way_fr.zip" # UN Parallel Corpus French (FR)

# corpus_source_folder = "/content/drive/MyDrive/"
# corpus_target_folder = "/tmp/"

# if (not os.path.exists(corpus_target_folder + EN_corpus_file)):
#   with zipfile.ZipFile(corpus_source_folder + EN_corpus_file,"r") as zip_ref:
#       zip_ref.extractall(corpus_target_folder)

# if (not os.path.exists(corpus_target_folder + ES_corpus_file)):
#   with zipfile.ZipFile(corpus_source_folder + ES_corpus_file,"r") as zip_ref:
#       zip_ref.extractall(corpus_target_folder)

# if (not os.path.exists(corpus_target_folder + FR_corpus_file)):
#   with zipfile.ZipFile(corpus_source_folder + FR_corpus_file,"r") as zip_ref:
#       zip_ref.extractall(corpus_target_folder)


In [ ]:
# Use the cuDNN backend for 16-bit training and inference for CUDA-enabled GPUs
# torch.backends.cudnn.benchmark =  True
# torch.backends.cudnn.enabled =  True

In [ ]:
torch.backends.cudnn.allow_tf32 = True

In [ ]:
max_seq_length = 64 # Re-calculated further 
min_seq_length = 16 # Re-calculated further 

In [ ]:

NUMBER_OF_EPOCHS = 1
EARLY_STOPPING_EPOCHS = 3
CLIPPING_VALUE = 15                   # clipping value, or None to prevent gradient clipping
MAXIMUM_NUMBER_OF_SAMPLES = 500000
BATCH_SIZE = 32

# Lets GPU to get more cooler
INNER_GPU_REST_TIME = 0 # Rest time between epochs
OUTER_GPU_REST_TIME = 0 # Rest time between model training / validation processes


In [ ]:
import pandas as pd  

def generate_examples(src_file, tgt_a_file, tgt_b_file):
    data = []
    with open(src_file, encoding="utf-8") as src_f, open(tgt_a_file, encoding="utf-8") as tgt_a, open(tgt_b_file, encoding="utf-8") as tgt_b:
        for idx, (a, b, c) in enumerate(zip(src_f, tgt_a, tgt_b)):           
            if (a.isspace() | b.isspace() | c.isspace()):
                continue
            data.append({'text_en' : a, 'text_es' : b, 'text_fr' :c} )
    return pd.DataFrame.from_records(data=data)


In [ ]:
import re
from unicodedata import normalize

PAD_WORD = '<pad>'
UNK_WORD = '<unk>'  # Unknown token symbol
BOS_WORD = '<bos>'  # Begin-of-Sentence token symbol
EOS_WORD = '<eos>'  # End-of-Sentence token symbol

def clean_text(text : str):
    text = normalize('NFD', str(text).lower())
    text = re.sub('[^A-Za-z ]+', '', text)
    return text

pkl_dataset_file = "transformers_us_ds.pkl"
un_ds = pd.DataFrame()

if os.path.exists(pkl_dataset_file):
    un_ds = pd.read_pickle(pkl_dataset_file) 
else:
    en_train_path = "./mnt/drive/UNv1.0.6way.en"
    es_train_path = "./mnt/drive/UNv1.0.6way.es"
    fr_train_path = "./mnt/drive/UNv1.0.6way.fr"
    df = generate_examples(en_train_path, es_train_path, fr_train_path)
    df['text_en'] = df['text_en'].apply(lambda row: clean_text(row))
    df['text_es'] = df['text_es'].apply(lambda row: clean_text(row))
    df['text_fr'] = df['text_fr'].apply(lambda row: clean_text(row))
    un_ds = df
    un_ds.to_pickle(pkl_dataset_file)
len(un_ds)


In [ ]:
un_ds.head(25)

In [ ]:
un_ds.head(-25)

In [ ]:
un_ds = un_ds.head(MAXIMUM_NUMBER_OF_SAMPLES)

In [ ]:
from enum import Enum

import torchtext

en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
es_tokenizer = get_tokenizer('spacy', language='es_core_news_sm')
fr_tokenizer = get_tokenizer('spacy', language='fr_core_news_sm')

Language = Enum('Language', ['EN', 'ES', 'FR'])

def yield_tokens(Lang: Language=Language.EN):
    for index, row in un_ds.iterrows():     
         if (Lang == Language.EN):
            yield en_tokenizer(str(row["text_en"]))  
         if (Lang == Language.ES):
            yield es_tokenizer(str(row["text_es"]))
         if (Lang == Language.FR):
           yield fr_tokenizer(str(row["text_fr"]))

       

In [ ]:
en_vocab_file = 'transformers_vocab_en.pth'
es_vocab_file = 'transformers_vocab_es.pth'
fr_vocab_file = 'transformers_vocab_fr.pth'

In [ ]:
import torchtext
import torchtext.data
import torchtext.data.datasets_utils
import torchtext.datasets

SPECIALS = [PAD_WORD, BOS_WORD, EOS_WORD,  UNK_WORD]

en_vocabulary = None
es_vocabulary = None
fr_vocabulary = None

if os.path.exists(en_vocab_file):
    en_vocabulary = torch.load(en_vocab_file)
else:
    en_vocabulary = build_vocab_from_iterator(yield_tokens(Language.EN), specials=SPECIALS)
    torch.save(en_vocabulary, en_vocab_file)

if os.path.exists(es_vocab_file):
    es_vocabulary = torch.load(es_vocab_file)
else:
    es_vocabulary = build_vocab_from_iterator(yield_tokens(Language.ES), specials=SPECIALS)
    torch.save(es_vocabulary, es_vocab_file)

if os.path.exists(fr_vocab_file):
    fr_vocabulary = torch.load(fr_vocab_file)
else:
    fr_vocabulary = build_vocab_from_iterator(yield_tokens(Language.FR), specials=SPECIALS)
    torch.save(fr_vocabulary, fr_vocab_file)

In [ ]:
print("EN Vocabulary Size = {0}", len(en_vocabulary))
print("ES Vocabulary Size = {0}", len(es_vocabulary))
print("FR Vocabulary Size = {0}", len(fr_vocabulary))

In [ ]:
PAD_EN_IDX = en_vocabulary[PAD_WORD]
BOS_EN_IDX = en_vocabulary[BOS_WORD]
EOS_EN_IDX = en_vocabulary[EOS_WORD]
UNK_EN_IDX = en_vocabulary[UNK_WORD]


PAD_ES_IDX = es_vocabulary[PAD_WORD]
BOS_ES_IDX = es_vocabulary[BOS_WORD]
EOS_ES_IDX = es_vocabulary[EOS_WORD]
UNK_ES_IDX = es_vocabulary[UNK_WORD]

PAD_FR_IDX = fr_vocabulary[PAD_WORD]
BOS_FR_IDX = fr_vocabulary[BOS_WORD]
EOS_FR_IDX = fr_vocabulary[EOS_WORD]
UNK_FR_IDX = fr_vocabulary[UNK_WORD]

en_vocab_size = len(en_vocabulary) + 1
es_vocab_size = len(es_vocabulary) + 1
fr_vocab_size = len(fr_vocabulary) + 1

In [ ]:
def pad_or_truncate(tokenized_text, allocate : bool = True,  pad_index : int = PAD_EN_IDX, bos_index : int = BOS_EN_IDX, eos_index : int = EOS_EN_IDX):  
    result = []
    if len(tokenized_text) < max_seq_length:
        if (allocate):
            result = [bos_index] + tokenized_text + [eos_index]
            left = max_seq_length - len(result)
            padding = [pad_index] * left
            result = result + padding
        else:
            left = max_seq_length - len(tokenized_text)
            padding = [pad_index] * left
            result = tokenized_text + padding       
    else:
        raise Exception("pad_or_truncate: max_seq_length not computed properly")
    return result

In [ ]:
full_data_file = 'transformers_full.pth'
trn_data_file = 'transformers_trn.pth'
val_data_file = 'transformers_val.pth'
tst_data_file = 'transformers_tst.pth'

In [ ]:
len(un_ds)

In [ ]:
import random
import pickle
from collections import Counter 

MINIMUM_ALLOWED_SIZE_OF_SEQ = 5
MAXIMUM_ALLOWED_SIZE_OF_SEQ = 100

full_data = []
trn_subset, val_subset, tst_subset = [], [], []
random.seed(1234)
torch.manual_seed(1234)

max_len = 0
min_len = 4500

en_counter = Counter()
es_counter = Counter()
fr_counter = Counter()

en_lengths = []
es_lengths = []
fr_lengths = []

fe = os.path.exists(full_data_file)
te = os.path.exists(trn_data_file)
ve = os.path.exists(val_data_file)
tt = os.path.exists(tst_data_file)

if fe and te and ve and tt:
    with open(full_data_file, 'rb') as fp:
            full_data = pickle.load(fp)
    with open(trn_data_file, 'rb') as fp:
            trn_subset = pickle.load(fp)
    with open(val_data_file, 'rb') as fp:
            val_subset = pickle.load(fp)        
    with open(tst_data_file, 'rb') as fp:
            tst_subset = pickle.load(fp)           
    for (en_seq,es_seq,fr_seq) in full_data:     
        en_counter.update(en_seq)
        es_counter.update(es_seq)
        fr_counter.update(fr_seq)
        a = len(en_seq)
        b = len(es_seq)
        c = len(fr_seq) 
        en_lengths.append(a)
        es_lengths.append(b)
        fr_lengths.append(c)
        max_len = max(max_len, a, b, c)    
        min_len = min(min_len, a, b, c)    
else:
    for idx, row in un_ds.iterrows():
        
        en_exp = row["text_en"].rstrip("\n")
        es_exp = row["text_es"].rstrip("\n")
        fr_exp = row["text_fr"].rstrip("\n")

        en_seq = [en_vocabulary[token] for token in en_tokenizer(en_exp)]
        es_seq = [es_vocabulary[token] for token in es_tokenizer(es_exp)]
        fr_seq = [fr_vocabulary[token] for token in fr_tokenizer(fr_exp)]

        a = len(en_seq)
        b = len(es_seq)
        c = len(fr_seq) 

        seq_min = min(a, b, c)
        seq_max = max(a, b, c)

        if seq_min  >= MINIMUM_ALLOWED_SIZE_OF_SEQ and seq_max <= MAXIMUM_ALLOWED_SIZE_OF_SEQ: 
            
            en_counter.update(en_seq)
            es_counter.update(es_seq)
            fr_counter.update(fr_seq)
       
            en_lengths.append(a)
            es_lengths.append(b)
            fr_lengths.append(c)

            max_len = max(max_len, seq_max)
            min_len = min(min_len, seq_min) 
            full_data.append((en_seq, es_seq, fr_seq))

    trn_subset, val_subset = train_test_split(full_data, test_size=0.3, train_size=0.7, random_state=1234, shuffle=True)
    val_subset, tst_subset = train_test_split(val_subset, test_size=0.33, train_size=0.67, random_state=1234, shuffle=True)

    with open(full_data_file, 'wb') as fp:
        pickle.dump(full_data, fp)
    with open(trn_data_file, 'wb') as fp:
        pickle.dump(trn_subset, fp)
    with open(val_data_file, 'wb') as fp:
        pickle.dump(val_subset, fp)
    with open(tst_data_file, 'wb') as fp:
        pickle.dump(tst_subset, fp)

if (max_seq_length < max_len):
    max_seq_length = max_len + 2

if (min_seq_length > min_len):
    min_seq_length = min_len + 2


print("Selected Records: {0}".format(len(full_data)))
print("MAX SEQ LEN {0}".format(max_seq_length))
print("MIN SEQ LEN {0}".format(min_seq_length))

size_of_trn_set_size = len(trn_subset)
size_of_val_set_size = len(val_subset)
size_of_tst_set_size = len(tst_subset)

print("Training tuples: {0} Validation tuples: {1} Testing tuples: {2}".format(size_of_trn_set_size, size_of_val_set_size, size_of_tst_set_size))


In [ ]:
fig = mp.figure(figsize=(8, 10))
ax0 = fig.add_subplot(311)
ax0.hist(en_lengths, rwidth=0.8, color='gray')
ax0.set_title("English Sentence Length")
ax0.set_xlabel("# Tokens in Sentence")

ax1 = fig.add_subplot(312)
ax1.hist(es_lengths, rwidth=0.8, color='gray')
ax1.set_title("Spanish Sentence Length")
ax1.set_xlabel("# Tokens in Sentence")

ax2 = fig.add_subplot(313)
ax2.hist(fr_lengths, rwidth=0.8, color='gray')
ax2.set_title("French Sentence Length")
ax2.set_xlabel("# Tokens in Sentence")

mp.tight_layout()
mp.show()

In [ ]:
mp.figure(figsize=(8,6))
mp.hist2d(en_lengths, es_lengths, bins=max_seq_length-2, cmap='binary')
mp.title("Joint Distribution of Sentence Lengths")
mp.xlabel("# English Tokens")
mp.ylabel("# Spanish Tokens")
mp.colorbar()
mp.show()

In [ ]:
mp.figure(figsize=(8,6))
mp.hist2d(en_lengths, fr_lengths, bins=max_seq_length-2, cmap='binary')
mp.title("Joint Distribution of Sentence Lengths")
mp.xlabel("# English Tokens")
mp.ylabel("# French Tokens")
mp.colorbar()
mp.show()

In [ ]:
def plot_top_words(counter : Counter, vocab: Vocab, k=20, ax=None):
    top_k = counter.most_common(k)
    tokens, freqs = zip(*reversed(top_k))
    
    words = [vocab.lookup_token(token) for token in tokens]


    if ax is None:
        mp.barh(words, freqs, color='gray')
    else:
        ax.barh(words, freqs, color='gray')

In [ ]:

fig = mp.figure(figsize=(14, 8))

ax0 = fig.add_subplot(131)
plot_top_words(en_counter, en_vocabulary, ax=ax0)
ax0.set_title("Top 20 English Words")
ax0.set_xlabel("Raw Frequency")

ax1 = fig.add_subplot(132)
plot_top_words(es_counter,es_vocabulary, ax=ax1)
ax1.set_title("Top 20 Spanish Words")
ax1.set_xlabel("Raw Frequency")

ax2 = fig.add_subplot(133)
plot_top_words(fr_counter,fr_vocabulary, ax=ax2)
ax2.set_title("Top 20 French Words")
ax2.set_xlabel("Raw Frequency")

mp.tight_layout()
mp.show()

In [ ]:
from  torch.nn.utils.rnn import pad_sequence

for en_id, es_id, fr_id in zip(en_vocabulary.lookup_indices(SPECIALS), es_vocabulary.lookup_indices(SPECIALS), fr_vocabulary.lookup_indices(SPECIALS)):
  assert en_id == es_id & es_id == fr_id


def tensor_transform(token_ids: List[int], bos_idx, eos_idx): # 57.1 segs
    list = [bos_idx] + token_ids + [eos_idx]                         
    return torch.as_tensor(list, device=DEVICE)

def generate_batch(data_batch):
    en_batch, es_batch, fr_batch = [], [], []
    for (en_item, es_item, fr_item) in data_batch:     
        en_t = tensor_transform(en_item, BOS_EN_IDX, EOS_EN_IDX)
        es_t = tensor_transform(es_item, BOS_ES_IDX, EOS_ES_IDX)
        fr_t = tensor_transform(fr_item, BOS_FR_IDX, EOS_FR_IDX)     
        en_batch.append(en_t)
        es_batch.append(es_t)
        fr_batch.append(fr_t)
    en_batch = pad_sequence(en_batch, padding_value=PAD_EN_IDX, batch_first=False)
    es_batch = pad_sequence(es_batch, padding_value=PAD_ES_IDX, batch_first=False)
    fr_batch = pad_sequence(fr_batch, padding_value=PAD_FR_IDX, batch_first=False)
    return en_batch, es_batch, fr_batch

def generate_batch_EN_ES(data_batch):
    en_batch, es_batch = [], []
    for (en_item, es_item, _) in data_batch:     
        en_t = tensor_transform(en_item, BOS_EN_IDX, EOS_EN_IDX)
        es_t = tensor_transform(es_item, BOS_ES_IDX, EOS_ES_IDX)
        en_batch.append(en_t)
        es_batch.append(es_t)
    en_batch = pad_sequence(en_batch, padding_value=PAD_EN_IDX, batch_first=False)
    es_batch = pad_sequence(es_batch, padding_value=PAD_ES_IDX, batch_first=False)
    return en_batch, es_batch

def generate_batch_EN_FR(data_batch):
    en_batch, fr_batch = [], []
    for (en_item, _, fr_item) in data_batch:        
        en_t = tensor_transform(en_item, BOS_EN_IDX, EOS_EN_IDX)
        fr_t = tensor_transform(fr_item, BOS_FR_IDX, EOS_FR_IDX)
        en_batch.append(en_t)
        fr_batch.append(fr_t)
    en_batch = pad_sequence(en_batch, padding_value=PAD_EN_IDX, batch_first=False)
    fr_batch = pad_sequence(fr_batch, padding_value=PAD_ES_IDX, batch_first=False)
    return en_batch, fr_batch

In [ ]:
from torch.utils.data import Dataset

class LanguageDataset(Dataset):
    
    def __init__(self, subset, length):
        self.length = length
        self.subset = subset

    def __getitem__(self, idx):
        return self.subset[idx]

    def __len__(self):
        return self.length

In [ ]:




Bilingual_EN_to_ES_trnset = DataLoader(LanguageDataset(trn_subset, size_of_trn_set_size), batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch_EN_ES)
Bilingual_EN_to_ES_valset = DataLoader(LanguageDataset(val_subset, size_of_val_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch_EN_ES)
Bilingual_EN_to_ES_tstset = DataLoader(LanguageDataset(tst_subset, size_of_tst_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch_EN_ES)

Bilingual_EN_to_FR_trnset = DataLoader(LanguageDataset(trn_subset, size_of_trn_set_size), batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch_EN_FR)
Bilingual_EN_to_FR_valset = DataLoader(LanguageDataset(val_subset, size_of_val_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch_EN_FR)
Bilingual_EN_to_FR_tstset = DataLoader(LanguageDataset(tst_subset, size_of_tst_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch_EN_FR)

Trilingual_trnset = DataLoader(LanguageDataset(trn_subset, size_of_trn_set_size), batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
Trilingual_valset = DataLoader(LanguageDataset(val_subset, size_of_val_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch)
Trilingual_tstset = DataLoader(LanguageDataset(tst_subset, size_of_tst_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch)

batches_for_training = math.ceil( size_of_trn_set_size / BATCH_SIZE)
batches_for_validation = math.ceil( size_of_val_set_size / BATCH_SIZE)


In [ ]:
import sys

size_to_MB = 1024 * 1024

def getSizeOf(a : torch.Tensor):
    return sys.getsizeof(a) + torch.numel(a) * a.element_size()

def ElementsOf(a : torch.Tensor):
    return torch.numel(a)
    

torch.cuda.empty_cache()


total = 0

for (src, tgt_ES, tgt_FR) in iter(Trilingual_trnset):

    total += getSizeOf(src) / size_to_MB
    total += getSizeOf(tgt_ES) / size_to_MB
    total += getSizeOf(tgt_FR) / size_to_MB

    del src
    del tgt_ES
    del tgt_FR

torch.cuda.empty_cache()

print("Tamaño estimado del conjunto de datos: {} Mbytes".format(total))

In [ ]:
def init_transformer_model(model : nn.Module):
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

In [ ]:
def masked_accuracy(label, pred, tgt_pad_idx = PAD_ES_IDX):
  pred = torch.argmax(pred, dim=-1)
  match = label.eq(pred)
  mask = label.ne(tgt_pad_idx)
  match = match & mask
  match = match.type(torch.float32) 
  mask =  mask.type(torch.float32)
  return torch.sum(match)/torch.sum(mask)

In [ ]:
import time

def let_gpu_rest(minutes):
    if minutes > 0:
        time.sleep(minutes * 60)    

In [ ]:
# Fragmento de código. Funciones que despliegan las curvas de desempeño para el modelo
# Se reutilizan a lo largo del ejercicio.

import matplotlib.pyplot as plt


PLOT_X = 10
PLOT_Y = 2.7    

def plot_accuracy_curve(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Exactitud del modelo {0}'.format(name))
    ax.plot(history['train_acc'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_acc'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_yticks(np.arange(0, 1, 1 / 10))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Exactitud')
    ax.legend()
    plt.show()

def plot_loss_curve(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Pérdida del modelo {0}'.format(name))
    ax.plot(history['train_loss'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_loss'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Pérdida')
    ax.legend()
    plt.show()

def plot_pplx_curve(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Perplejidad del modelo {0}'.format(name))
    ax.plot(history['train_ppl'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_ppl'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Perplejidad')
    ax.legend()
    plt.show()


def plot_accuracy_curve_dual_transformer(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Exactitud del modelo {0}'.format(name))
    ax.plot(history['train_conjugate_acc'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_conjugate_acc'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_yticks(np.arange(0, 1, 1 / 10))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Exactitud')
    ax.legend()
    plt.show()


def plot_loss_curve_dual_transformer(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Pérdida del modelo {0}'.format(name))
    ax.plot(history['train_conjugate_loss'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_conjugate_loss'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Pérdida')
    ax.legend()
    plt.show()

def plot_accuracy_curve_dual_transformer_both(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Exactitud del modelo {0}'.format(name))
    ax.plot(history['train_acc_a'], label='Entrenamiento (EN-ES)')
    ax.plot(history['train_acc_b'], label='Entrenamiento (EN-FR)')
    ax.plot(history['valid_acc_a'], label='Validación (EN-ES)')
    ax.plot(history['valid_acc_b'], label='Validación (EN-FR)')
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_yticks(np.arange(0, 1, 1 / 10))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Exactitud')
    ax.legend()
    plt.show()

def plot_loss_curve_dual_transformer_both(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Pérdida del modelo {0}'.format(name))
    ax.plot(history['train_loss_a'], label='Entrenamiento (EN-ES)')
    ax.plot(history['train_loss_b'], label='Entrenamiento (EN-FR)')
    ax.plot(history['valid_loss_a'], label='Validación (EN-ES)')
    ax.plot(history['valid_loss_b'], label='Validación (EN-FR)')
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Pérdida')
    ax.legend()
    plt.show()


# Seq-to-Seq using RNN Layers

In [ ]:
RNN_Learning_Rate = 1e-4

ENCODER_EMBEDDING_DIM = 256
DECODER_EMBEDDING_DIM = 256
ENCODER_HIDDEN_SIZE = 512
DECODER_HIDDEN_SIZE = 512

encoder_dropout = 0.2 
decoder_dropout = 0.2

RNN_CLIPPING_VALUE = 10

In [ ]:
RNN_EN_ES_model_path = 'RNN_en_es_{0}.pt'
RNN_EN_FR_model_path = 'RNN_en_fr_{0}.pt'

### Model building blocks

In [ ]:
# Based on https://github.com/CRCTransformers/deepdive-book/

class BahdanauEncoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, dropout_p):
        super().__init__()
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim
        self.encoder_hidden_dim = encoder_hidden_dim
        self.decoder_hidden_dim = decoder_hidden_dim
        self.dropout_p = dropout_p
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.gru = nn.GRU(embedding_dim, encoder_hidden_dim,  bidirectional=True)
        self.linear = nn.Linear(encoder_hidden_dim * 2, decoder_hidden_dim)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        outputs, hidden = self.gru(embedded)
        hidden = torch.tanh(self.linear(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)))
        return outputs, hidden

In [ ]:
# Based on https://github.com/CRCTransformers/deepdive-book/
    
class BahdanauAttentionQKV(nn.Module):

    def __init__(self, hidden_size, query_size=None, key_size=None, dropout_p=0.15):
        super().__init__()
        self.hidden_size = hidden_size
        self.query_size = hidden_size if query_size is None else query_size
        # assume bidirectional encoder, but can specify otherwise
        self.key_size = 2 * hidden_size if key_size is None else key_size
        self.query_layer = nn.Linear(self.query_size, hidden_size)
        self.key_layer = nn.Linear(self.key_size, hidden_size)
        self.energy_layer = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, hidden, encoder_outputs, src_mask=None):
        # (B, H)
        query_out = self.query_layer(hidden)
        # (Src, B, 2*H) --> (Src, B, H)
        key_out = self.key_layer(encoder_outputs)
        # (B, H) + (Src, B, H) = (Src, B, H)
        energy_input = torch.tanh(query_out + key_out)
        # (Src, B, H) --> (Src, B, 1) --> (Src, B)
        energies = self.energy_layer(energy_input).squeeze(2)
        # if a mask is provided, remove masked tokens from softmax calc
        if src_mask is not None:
            energies.data.masked_fill_(src_mask == 0, float("-inf"))
        # softmax over the length dimension
        weights = F.softmax(energies, dim=0)
        # return as (B, Src) as expected by later multiplication
        return weights.transpose(0, 1)

In [ ]:
# Based on https://github.com/CRCTransformers/deepdive-book/
    
class BahdanauDecoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, attention, dropout_p):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.output_dim = output_dim
        self.encoder_hidden_dim = encoder_hidden_dim
        self.decoder_hidden_dim = decoder_hidden_dim
        self.dropout_p = dropout_p
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.attention = attention # allowing for custom attention
        self.gru = nn.GRU((encoder_hidden_dim * 2) +  embedding_dim, decoder_hidden_dim)
        self.out = nn.Linear((encoder_hidden_dim * 2) +  embedding_dim + decoder_hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input, hidden, encoder_outputs, src_mask=None):
        # (B) --> (1, B)
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        attentions = self.attention(hidden, encoder_outputs, src_mask)
        # (B, S) --> (B, 1, S)
        a = attentions.unsqueeze(1)
        # (S, B, 2*Enc) --> (B, S, 2*Enc)
        encoder_outputs = encoder_outputs.transpose(0, 1)
        # weighted encoder representation
        # (B, 1, S) @ (B, S, 2*Enc) = (B, 1, 2*Enc)
        weighted = torch.bmm(a, encoder_outputs)
        # (B, 1, 2*Enc) --> (1, B, 2*Enc)
        weighted = weighted.transpose(0, 1)
        # concat (1, B, Emb) and (1, B, 2*Enc)
        # results in (1, B, Emb + 2*Enc)
        rnn_input = torch.cat((embedded, weighted), dim=2)
        output, hidden = self.gru(rnn_input, hidden.unsqueeze(0))
        assert (output == hidden).all()
        # get rid of empty leading dimensions
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        # concatenate the pieces above
        # (B, Dec), (B, 2*Enc), and (B, Emb)
        # result is (B, Dec + 2*Enc + Emb)
        linear_input = torch.cat((output, weighted, embedded), dim=1)
        # (B, Dec + 2*Enc + Emb) --> (B, O)
        output = self.out(linear_input)
        return output, hidden.squeeze(0), attentions

In [ ]:
# Based on https://github.com/CRCTransformers/deepdive-book/
    
class MultipleOptimizer(object):
    def __init__(self, optimizers : List):
        self.optimizers = optimizers

    def zero_grad(self):
        for op in self.optimizers:
            op.zero_grad()

    def step(self):
        for op in self.optimizers:
            op.step()
    
    def __getitem__(self, index):
        return self.optimizers[index]

### 1-Encode 1-Decoder Seq2Seq using RNN

In [ ]:
# Based on https://github.com/CRCTransformers/deepdive-book/
    
class Seq2Seq(nn.Module):
    def __init__(self, name, encoder, decoder, device=None):
        super().__init__()
        self.name = name
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.tgt_vocab_size = decoder.output_dim

    def forward(self, src, tgt, src_mask=None, teacher_forcing_ratio=0.5, return_attentions=False):

        tgt_length, batch_size = tgt.shape
        
        encoder_outputs, hidden = self.encoder(src)

        hidden = hidden.squeeze(1) # B, 1, Enc --> B, Enc (if necessary)

        
        # store decoder outputs
        outputs = torch.zeros(tgt_length, batch_size, self.tgt_vocab_size, device=self.device)

        # start with  as the decoder input
        decoder_input = tgt[0, :]
        attentions = []
        for t in range(1, tgt_length):
            decoder_output, hidden, attention = self.decoder(decoder_input, hidden, encoder_outputs, src_mask)
            outputs[t] = decoder_output
            teacher_force = random.random() < teacher_forcing_ratio
            top_token = decoder_output.max(1)[1]
            decoder_input = (tgt[t] if teacher_force else top_token)
            attentions.append(attention.unsqueeze(-1))
     
    
        if return_attentions:
            return outputs, torch.cat(attentions, dim=-1)
        else:
            return outputs


    def getName(self):
        return self.name  

### RNN Model train and evaluate routines

In [ ]:
gc_count = 0

def train_single_rnn(model,  dataset, optimizer, loss_fn, src_pad_idx, tgt_pad_idx, device, clip=None, smoothing = True):
    model.train()       
    epoch_loss = 0
    epoch_accm = 0 
    epoch_wmem = 0
    epoch_lmem = 0   
    with tqdm(total=batches_for_training, leave=False) as t:
        i = 0
        batches_before_gc = 0
        for  src, tgt in iter(dataset):
           
            msk = (src != PAD_ES_IDX)

            epoch_wmem += getSizeOf(src)
            epoch_wmem += getSizeOf(tgt)
            epoch_wmem += getSizeOf(msk)

            optimizer.zero_grad()                 

            logits =  model(src, tgt, msk, teacher_forcing_ratio=0.5)

            loss = 0
            accm = 0

            epoch_lmem += getSizeOf(logits)

            
            pv = logits[1:].view(-1, logits.shape[2])
            gt = tgt[1:].view(-1)

            loss = loss_fn(pv, gt)
            loss.backward()
        
            if (not clip == None):
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            
            optimizer.step() 
                
            accm = masked_accuracy(gt, pv,  PAD_ES_IDX)
        
            epoch_loss += loss
            epoch_accm += accm
             
            t.update()
            epoch_wmem += getSizeOf(loss)
            epoch_wmem += getSizeOf(accm)
            i += 1
       
            del src
            del tgt
            del msk
            del logits
            del pv
            del gt
            del loss
            del accm
            batches_before_gc += 1
    
            if (batches_before_gc > gc_count):
                torch.cuda.empty_cache()       
                batches_before_gc = 0
            if i > 5:
                break

    train_loss = (epoch_loss / batches_for_training).item()
    train_accm = (epoch_accm / batches_for_training).item() 
    wrk_mem = epoch_wmem / batches_for_training
    log_mem = epoch_lmem / batches_for_training    
    return train_loss, train_accm, wrk_mem, log_mem
    
def eval_single_rnn(model,  dataset, loss_fn, src_pad_idx, tgt_pad_idx, device):
    model.eval()    
    epoch_loss = 0
    epoch_accm = 0 
    epoch_wmem = 0
    epoch_lmem = 0  
    with torch.no_grad():
        with tqdm(total=batches_for_validation, leave=False) as t:       
            i = 0
            batches_before_gc = 0
            for  src, tgt in iter(dataset):

                msk = (src != PAD_ES_IDX)

                epoch_wmem += getSizeOf(src)
                epoch_wmem += getSizeOf(tgt)
                epoch_wmem += getSizeOf(msk)
            
                logits =  model(src, tgt, msk,  teacher_forcing_ratio=0.0)

                loss = 0
                accm = 0

                epoch_lmem += getSizeOf(logits)

                pv = logits[1:].view(-1, logits.shape[2])
                gt = tgt[1:].view(-1)

                loss = loss_fn(pv, gt)
                accm = masked_accuracy(gt, pv,  PAD_ES_IDX)

                epoch_loss += loss              
                epoch_accm += accm


                epoch_wmem += getSizeOf(loss)
                epoch_wmem += getSizeOf(accm)
                t.update()
                i += 1
 
                del src
                del tgt
                del msk
                del logits
                del pv
                del gt
                del loss
                del accm
                batches_before_gc += 1
                if (batches_before_gc > gc_count):
                    torch.cuda.empty_cache()       
                    batches_before_gc = 0
                if i > 5:
                    break
    
    valid_loss = (epoch_loss / batches_for_validation).item()
    valid_accm = (epoch_accm / batches_for_validation).item()
    wrk_mem = epoch_wmem / batches_for_validation
    log_mem = epoch_lmem / batches_for_validation
    return valid_loss , valid_accm, wrk_mem, log_mem

In [ ]:
import cProfile, pstats, io
from pstats import SortKey

from timeit import default_timer as timer

def run_single_rnn(model, filename, trnset, valset, optimizer, loss_fn, src_pad_idx, tgt_pad_idx):
    
    
    print(model.getName())    
    
    pr = cProfile.Profile()
    pr.enable()


    metrics_by_epoch = {}
    
    best_valid_loss = float("inf")
    
    early_stopping_count = 0

    base_epoch = -1 
    # Check for previos training checkpoints (latest)
    for epoch in range(NUMBER_OF_EPOCHS):
        if os.path.exists(filename.format(epoch)):
            base_epoch = epoch
        else:
            break
    
    # Load the latest checkpoint 
    if (base_epoch >= 0):
        # Load model checkpoint file (latest)
        checkpoint = torch.load(filename.format(base_epoch))
        # Load model data only when match filename with epoch information
        if (checkpoint['epoch'] == base_epoch):
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer[0].load_state_dict(checkpoint['optimizer_enc_state_dict'])
            optimizer[1].load_state_dict(checkpoint['optimizer_dec_state_dict'])
            metrics_by_epoch = checkpoint['metrics_by_epoch']
            best_valid_loss = checkpoint['best_valid_loss']
            early_stopping_count = checkpoint['early_stopping_count']
            print("Continuing training model {0} from checkpoint at epoch {1}".format(model.getName(), base_epoch + 1))
        else:
            raise RuntimeError("Checkpoint {0} is corrupt".format(base_epoch))

    if (base_epoch + 1 < NUMBER_OF_EPOCHS):
        for epoch in tqdm(range(base_epoch + 1, NUMBER_OF_EPOCHS), desc="Epoch"):

            

            start_time = timer()      
            train_loss, train_acc, trn_wrk_mem, trn_log_mem = train_single_rnn(model, trnset, optimizer, loss_fn,  src_pad_idx, tgt_pad_idx, DEVICE, clip=CLIPPING_VALUE)
            end_time = timer()
           
               
            valid_loss, valid_acc, val_wrk_mem, val_log_mem = eval_single_rnn(model, valset, loss_fn, src_pad_idx, tgt_pad_idx, DEVICE)
            seconds = end_time - start_time

            tqdm.write("Model {0} at Epoch {1} Trn.Loss = {2} and Trn.Accuracy = {3}".format(model.getName(), epoch + 1, train_loss, train_acc))
            tqdm.write("Model {0} at Epoch {1} Val.Loss = {2} and Val.Accuracy = {3}".format(model.getName(), epoch + 1, valid_loss, valid_acc))

            metrics_by_epoch[epoch + 1] = dict(
                model_name = model.getName(),
                model_epoch = epoch + 1,
                train_loss = train_loss,
                train_acc = train_acc,
                train_ppl = np.exp(train_loss),
                train_wrkmem = trn_wrk_mem,
                train_logmem = trn_log_mem,
                valid_loss = valid_loss,
                valid_acc = valid_acc,
                valid_ppl = np.exp(valid_loss),
                valid_wrkmem = val_wrk_mem,
                valid_logmem = val_log_mem,
                time = seconds
            )


            temp_filename = "temp_" + filename.format(epoch)

            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_enc_state_dict': optimizer[0].state_dict(),
            'optimizer_dec_state_dict': optimizer[1].state_dict(),
            'early_stopping_count': early_stopping_count,
            'metrics_by_epoch' : metrics_by_epoch,
            'best_valid_loss' : best_valid_loss
            }, temp_filename)

            os.rename(temp_filename, filename.format(epoch))

            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                early_stopping_count = 0
            elif epoch > EARLY_STOPPING_EPOCHS:
                early_stopping_count += 1
                      
            if early_stopping_count == EARLY_STOPPING_EPOCHS:
                tqdm.write(f"Early stopping triggered in epoch {epoch + 1}")
                break
        
            tqdm.write("Waiting for gpu cooling time...")

            let_gpu_rest(INNER_GPU_REST_TIME)

    pr.disable()
    s = io.StringIO()
    sortby = SortKey.CUMULATIVE
    ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
    ps.print_stats()
    print(s.getvalue())


    return metrics_by_epoch

### Model testing

In [ ]:

enc_ES = BahdanauEncoder(input_dim=en_vocab_size, embedding_dim=ENCODER_EMBEDDING_DIM, encoder_hidden_dim=ENCODER_HIDDEN_SIZE, decoder_hidden_dim=DECODER_HIDDEN_SIZE, dropout_p = encoder_dropout)
enc_FR = BahdanauEncoder(input_dim=en_vocab_size, embedding_dim=ENCODER_EMBEDDING_DIM, encoder_hidden_dim=ENCODER_HIDDEN_SIZE, decoder_hidden_dim=DECODER_HIDDEN_SIZE, dropout_p = encoder_dropout)

attn_ES = BahdanauAttentionQKV(DECODER_HIDDEN_SIZE)
attn_FR = BahdanauAttentionQKV(DECODER_HIDDEN_SIZE)

dec_ES = BahdanauDecoder(output_dim=es_vocab_size, embedding_dim=DECODER_EMBEDDING_DIM, encoder_hidden_dim=ENCODER_HIDDEN_SIZE, decoder_hidden_dim=DECODER_HIDDEN_SIZE, attention=attn_ES, dropout_p=decoder_dropout)
dec_FR = BahdanauDecoder(output_dim=fr_vocab_size, embedding_dim=DECODER_EMBEDDING_DIM, encoder_hidden_dim=ENCODER_HIDDEN_SIZE, decoder_hidden_dim=DECODER_HIDDEN_SIZE, attention=attn_FR, dropout_p=decoder_dropout)

seq2seq_EN_ES = Seq2Seq("RNN_EN_ES", enc_ES, dec_ES, DEVICE)
seq2seq_EN_FR = Seq2Seq("RNN_EN_FR", enc_FR, dec_FR, DEVICE)



RNN_enc_optim_EN_ES = torch.optim.AdamW(seq2seq_EN_ES.encoder.parameters(),lr=RNN_Learning_Rate)
RNN_dec_optim_EN_ES = torch.optim.AdamW(seq2seq_EN_ES.decoder.parameters(),lr=RNN_Learning_Rate)

RNN_enc_optim_EN_FR = torch.optim.AdamW(seq2seq_EN_FR.decoder.parameters(),lr=RNN_Learning_Rate)
RNN_dec_optim_EN_FR = torch.optim.AdamW(seq2seq_EN_FR.decoder.parameters(),lr=RNN_Learning_Rate)

loss_fn_RNN_EN_ES = torch.nn.CrossEntropyLoss(ignore_index=PAD_ES_IDX)
loss_fn_RNN_EN_FR = torch.nn.CrossEntropyLoss(ignore_index=PAD_FR_IDX)

optims_EN_ES = MultipleOptimizer([RNN_enc_optim_EN_ES, RNN_dec_optim_EN_ES])
optims_EN_FR = MultipleOptimizer([RNN_enc_optim_EN_FR, RNN_dec_optim_EN_FR])


In [ ]:
random.seed(1234)
torch.manual_seed(1234)
seq2seq_EN_ES.apply(init_transformer_model)

In [ ]:
count_params(seq2seq_EN_ES)

In [ ]:
random.seed(1234)
torch.manual_seed(1234)
seq2seq_EN_FR.apply(init_transformer_model)

In [ ]:
count_params(seq2seq_EN_FR)

In [58]:

histories = []


import cProfile, pstats, io



random.seed(1234)
torch.manual_seed(1234)
seq2seq_EN_ES = seq2seq_EN_ES.to(DEVICE)
history = run_single_rnn(seq2seq_EN_ES, RNN_EN_ES_model_path, Bilingual_EN_to_ES_trnset, Bilingual_EN_to_ES_valset, optims_EN_ES, loss_fn_RNN_EN_ES, PAD_EN_IDX, PAD_ES_IDX)
name = seq2seq_EN_ES.getName()
df = pd.DataFrame.from_dict(history, orient='index')   
plot_accuracy_curve(name, df)
plot_loss_curve(name, df)
plot_pplx_curve(name, df)
histories.append(df)
torch.cuda.empty_cache()
let_gpu_rest(OUTER_GPU_REST_TIME)



random.seed(1234)
torch.manual_seed(1234)
seq2seq_EN_FR = seq2seq_EN_FR.to(DEVICE)
history = run_single_rnn(seq2seq_EN_FR, RNN_EN_FR_model_path, Bilingual_EN_to_FR_trnset, Bilingual_EN_to_FR_valset, optims_EN_FR, loss_fn_RNN_EN_FR, PAD_EN_IDX, PAD_FR_IDX)
name = seq2seq_EN_FR.getName()
df = pd.DataFrame.from_dict(history, orient='index')   
plot_accuracy_curve(name, df)
plot_loss_curve(name, df)
plot_pplx_curve(name, df)
histories.append(df)
torch.cuda.empty_cache()
let_gpu_rest(OUTER_GPU_REST_TIME)


result = pd.concat(histories)
result.to_csv("./metrics_of_rnn_models.csv", sep=';', index=True, encoding='utf-8')

display(result)